# Mistral and Weights & Biases

In this notebooks you will learn how to trace your MistralAI Api calls using W&B Weave, how to evaluate the performance of your models and how to close the gap by leveraging the MistralAI finetuning capabilities.

- Weights & Biases: https://wandb.ai/
- Mistral finetuning docs: https://docs.mistral.ai/capabilities/finetuning/
- Tracing with W&B Weave: https://wandb.me/weave

In [ ]:
# !pip install mistralai pandas weave

## Using Mistral and Weave

You will probably integrate MistralAI API calls in your codebase by creating a function like the one below:

In [ ]:
import os, asyncio, json
import weave
from mistralai.async_client import MistralAsyncClient
from mistralai.models.chat_completion import ChatMessage

client = MistralAsyncClient(api_key=os.environ["MISTRAL_API_KEY"])

@weave.op()  # <---- add this and you are good to go
async def call_mistral(model:str, messages:list, **kwargs) -> str:
    "Call the Mistral API"
    chat_response = await client.chat(
        model=model,
        messages=messages,
        **kwargs,
    )
    return chat_response.choices[0].message.content

The only thing you need to do is add the @weave.op() decorator to the function you want to trace.

Let's define a more interesting function that recommends cheese based on the region and model.



In [ ]:
@weave.op()
async def cheese_recommender(region:str, model:str) -> str:
    "Recommend the best cheese in a given region"
     
    messages = [ChatMessage(
        role="user", 
        content=f"What is the best cheese in {region}?")]

    cheeses = await call_mistral(model=model, messages=messages)
    return {"region": region, "cheeses": cheeses}

Let's run this function and see how weave traces it. We call weave.init() to tell weave the project where to store the traces.

In [ ]:

weave.init("mistral_webinar")
out = await cheese_recommender(region="France", model="open-mistral-7b")
print(out)

You can view the traces by clicking the link above 👆
![](cheese_recomender.png)



## Prepare the dataset

Weave also has Dataset support, so you can keep your data and the model outputs in the same place. You can convert alsmot any iterable into a dataset!

Let's load some Q/A data from our support [wandbot](https://github.com/wandb/wandbot)



In [ ]:
import pandas as pd
df = pd.read_json('qa.jsonl', orient='records', lines=True)
df.head()

Let's split into train/valid

In [ ]:
df_train=df.sample(frac=0.9,random_state=200)
df_eval=df.drop(df_train.index)
len(df_train), len(df_eval)

In [ ]:
ds_train = weave.Dataset(name="ds_train", rows=df_train)
ds_eval = weave.Dataset(name="ds_eval", rows=df_eval)

let's publish them to Weave

In [ ]:
weave.publish(ds_train)
weave.publish(ds_eval)

![](dataset.png)

A neat trick to get better answers is instead of passing a very long initial message, passing a small conversation with some prefilled agent responses.

In [ ]:
def create_messages(question: str, cls=ChatMessage):
    messages = [
        cls(
            role="user", 
            content=(
                "You are an expert about Weights & Biases the ML platform. "
                 "You will answer questions about the product, Answer the question directly, without repeating the instructions."
                 )
        ),
        cls(
            role="assistant", 
            content=(
                "Sure, I'd be happy to help with your question about Weights & Biases. "
                 "If you have a specific question about using Weights & Biases, such as how to track experiments, "
                 "visualize data, or manage artifacts, please feel free to ask!")
        ),
        cls(
            role="user", 
            content=f"Here is the question: {question}"
        )
    ]
    return messages

In [ ]:
@weave.op()
async def wandb_expert(question:str, model:str) -> str:
    "Answer questions about wandb"
     
    messages = create_messages(question=question)

    answer = await call_mistral(model=model, messages=messages)
    return {"question": question, "answer": answer}

res = await wandb_expert(question=df.loc[0].question, model="mistral-medium-latest")
print(df.loc[0].question)
print(res["answer"])

## GT dataset
Let's create a dataset with mistral-medium-latest as our baseline

In [ ]:
class MistralModel(weave.Model):
    model: str
    temperature: float = 0.7
    
    @weave.op
    def create_messages(self, question:str):
        return create_messages(question)

    @weave.op
    async def predict(self, question:str):
        messages = self.create_messages(question)
        return await call_mistral(model=self.model, messages=messages)

Lets create a dataset with the medium model predictions

In [ ]:
mistral_medium = MistralModel(model="mistral-medium-latest")

In [ ]:
ds_eval

In [ ]:
async def async_foreach(sequence, func, max_concurrent_tasks):
    "Handy parallelism async for looper"
    semaphore = asyncio.Semaphore(max_concurrent_tasks)
    async def process_item(item):
        async with semaphore:
            result = await func(item)
            return item, result

    tasks = [asyncio.create_task(process_item(item)) for item in sequence]

    for task in asyncio.as_completed(tasks):
        item, result = await task
        yield item, result
        
async def map(ds, func, max_concurrent_tasks = 7, col_name="model_preds"):
    new_dataset = []
    async for example, map_results in async_foreach(ds.rows, func, max_concurrent_tasks):
        example.update({col_name: map_results})
        new_dataset.append(example)
    return new_dataset

ds_eval_medium_rows = await map(ds_eval, mistral_medium.predict, col_name="mistral_medium")

In [ ]:
ds_eval_medium = weave.Dataset(name="ds_eval_medium", description="Mistral medium predictions", rows=ds_eval_medium_rows)
weave.publish(ds_eval_medium)

You can pull your data back easily using the API:

In [ ]:
ds_eval_medium = weave.ref('ds_eval_medium:latest').get()

Let's add the results of Mistral 7B (non finetuned)

In [ ]:
mistral_7b = MistralModel(model="open-mistral-7b")
ds_eval_7b_rows = await map(ds_eval_medium, mistral_7b.predict, col_name="mistral_7b")
ds_eval_7b_medium = weave.Dataset(name="ds_eval_medium_7b", description="Mistral 7b predictions along with medium", rows=ds_eval_7b_rows)
weave.publish(ds_eval_7b_medium)

![](medium_7b.png)

## Evaluation
Let's use mistral large as a judge, let's compute a score as baseline comparing `7B` and `medium`.


In [ ]:
class LLMJudge(weave.Model):
    model: str = "mistral-large-latest"
    
    @weave.op
    async def predict(self, question: str, mistral_7b: str, mistral_medium: str, answer: str, **kwargs) -> dict:
        messages = [
            ChatMessage(
                role="user",
                content=(
                "You are an expert about Weights & Biases the ML platform. "
                "You have to pick the best answer between two answers. "
                "Take into consideration the context of the question and the ground truth answer as a reference. \n"
                "Here is the question: {question}\n"
                "Here is the answer1: {mistral_7b}\n"
                "Here is the answer2: {mistral_medium}\n"
                "Ground truth answer: {answer}\n"
                "Return the name of the best_answer (or None if you think both are wrong) and the reason in short JSON object.").format(
                    question=question, 
                    mistral_7b=mistral_7b, 
                    mistral_medium=mistral_medium,
                    answer=answer)
            )
        ]
        payload = await call_mistral(model=self.model, messages=messages, response_format={"type": "json_object"})
        return json.loads(payload)

In [ ]:
ds_eval_7b_medium.rows[0].keys()

In [ ]:
llm_judge = LLMJudge()
res = await llm_judge.predict(**ds_eval_7b_medium.rows[0])
res

In [ ]:
@weave.op
def evaluate_answer(model_output: str) -> dict:
    "Evaluate the answer"
    return {"win": model_output["best_answer"] == "answer1"}

Let's define a weave.evaluation

In [ ]:
evaluation = weave.Evaluation(dataset=ds_eval_7b_medium, scorers=[evaluate_answer])

In [ ]:
await evaluation.evaluate(llm_judge)

![](eval_base.png)

## Fine-Tune FTW

This is pretty descent for both 😍. Let's see if fine-tuning improves this.

In [ ]:
def format_messages(row):
    "Format on the expected MistralAI fine-tuning dataset"
    question = row['question']
    answer = row['answer']
    messages = create_messages(question, cls=dict)
    # we need to append the answer for training 👇
    messages = {"messages":messages + [dict(role="assistant", content=answer)]}
    return messages

In [ ]:
msgs = format_messages(df_train.iloc[0])
msgs

In [ ]:
formatted_df_train = df_train.apply(format_messages, axis=1)
formatted_df_eval = df_eval.apply(format_messages, axis=1)
formatted_df_train.head()

In [ ]:
formatted_df_train.to_json("formatted_df_train.jsonl", orient="records", lines=True)
formatted_df_eval.to_json("formatted_df_eval.jsonl", orient="records", lines=True)

## Upload dataset

In [ ]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("formatted_df_train.jsonl", "rb") as f:
    ds_train = client.files.create(file=("formatted_df_train.jsonl", f))
with open("formatted_df_eval.jsonl", "rb") as f:
    ds_eval = client.files.create(file=("eval.jsonl", f))


In [ ]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [ ]:
pprint(ds_train)

In [ ]:
pprint(ds_eval)

## Create a fine-tuning job

In [ ]:
from mistralai.models.jobs import TrainingParameters, WandbIntegrationIn

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[ds_train.id],
    validation_files=[ds_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=25,
        learning_rate=0.0001,
        ),
    integrations=[
        WandbIntegrationIn(
            project="mistral_webinar",
            run_name="finetune_wandb",
            api_key=os.environ.get("WANDB_API_KEY"),
        ).dict()
    ],
)

In [ ]:
pprint(created_jobs)

In [ ]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)



We can follow the training progress using in the wandb dashboard

![](ft.png)



In [ ]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

Let's retrieve the fie-tuned model. NOw we don't need to do any aditional setup, we can just use the model served for us using the MistralAI API

In [ ]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)


## Use a fine-tuned model

Let's compute the predictions using the fine-tuned 7B model

In [ ]:
ds_eval_medium = weave.ref('ds_eval_medium:latest').get()

Let's add the results of Mistral 7B-finetuned

In [ ]:
mistral_7b_ft = MistralModel(model=retrieved_jobs.fine_tuned_model)
ds_eval_7b_rows = await map(ds_eval_medium, mistral_7b_ft.predict, col_name="mistral_7b")
ds_eval_7b_ft_medium = weave.Dataset(name="ds_eval_medium_7b_ft", description="Finetuned Mistral 7b predictions along with medium", rows=ds_eval_7b_rows)
weave.publish(ds_eval_7b_ft_medium)

In [ ]:
evaluation = weave.Evaluation(dataset=ds_eval_7b_ft_medium, scorers=[evaluate_answer])

In [ ]:
await evaluation.evaluate(llm_judge)